## Language modelling with LSTMs

Now, we will be using LSTMs for language modelling by learning sequences of sentences (windows) that lead up to a certain word.

Note:
first time running, you will need to install ``tensorflow`` and ``keras``

In [1]:
# first time running, run the following comment to install tenserflow
!pip install tensorflow

You should consider upgrading via the 'c:\users\andre\anaconda3\python.exe -m pip install --upgrade pip' command.


In [2]:
# first time running, run the following comment to install keras
!pip install keras

You should consider upgrading via the 'c:\users\andre\anaconda3\python.exe -m pip install --upgrade pip' command.


In [3]:
import nltk
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import Dropout
from tensorflow.keras.utils import to_categorical
from keras.models import Model
from tensorflow.keras.layers import BatchNormalization
from keras.preprocessing import sequence
from keras.layers import Input
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from sklearn.model_selection import train_test_split

We use Emma by Jane Austen from the Gutenberg corpus again:

In [4]:
emma_sents = nltk.corpus.gutenberg.sents('austen-emma.txt')

In [5]:
# print the first 5 sentences
print(emma_sents[:5])

[['[', 'Emma', 'by', 'Jane', 'Austen', '1816', ']'], ['VOLUME', 'I'], ['CHAPTER', 'I'], ['Emma', 'Woodhouse', ',', 'handsome', ',', 'clever', ',', 'and', 'rich', ',', 'with', 'a', 'comfortable', 'home', 'and', 'happy', 'disposition', ',', 'seemed', 'to', 'unite', 'some', 'of', 'the', 'best', 'blessings', 'of', 'existence', ';', 'and', 'had', 'lived', 'nearly', 'twenty', '-', 'one', 'years', 'in', 'the', 'world', 'with', 'very', 'little', 'to', 'distress', 'or', 'vex', 'her', '.'], ['She', 'was', 'the', 'youngest', 'of', 'the', 'two', 'daughters', 'of', 'a', 'most', 'affectionate', ',', 'indulgent', 'father', ';', 'and', 'had', ',', 'in', 'consequence', 'of', 'her', 'sister', "'", 's', 'marriage', ',', 'been', 'mistress', 'of', 'his', 'house', 'from', 'a', 'very', 'early', 'period', '.']]


Next, we store all the words as integers (like a dictionary, similar to the one-hot vector):

In [6]:
# stores the words -> integers
word_dict = {}
# stores the integers -> words
reverse_dict = {}

# keeps track of the words
no_words = 1

int_sents = []
longest = 0
for i, sentence in enumerate(emma_sents[:1000]):
    ints = []
    
    for word in sentence:
        if word not in word_dict.keys():
            word_dict[word] = no_words
            reverse_dict[no_words] = word
            no_words += 1
        # add word to the integer list
        ints.append(word_dict[word])
    
    # Print to illustrate the conversion
    if i<5:
        print(sentence)
        print(ints)
    # Keep track of the longest sentence
    if len(sentence) > longest:
        longest = len(sentence)
        
    # store the integer sentence
    int_sents.append(ints)

['[', 'Emma', 'by', 'Jane', 'Austen', '1816', ']']
[1, 2, 3, 4, 5, 6, 7]
['VOLUME', 'I']
[8, 9]
['CHAPTER', 'I']
[10, 9]
['Emma', 'Woodhouse', ',', 'handsome', ',', 'clever', ',', 'and', 'rich', ',', 'with', 'a', 'comfortable', 'home', 'and', 'happy', 'disposition', ',', 'seemed', 'to', 'unite', 'some', 'of', 'the', 'best', 'blessings', 'of', 'existence', ';', 'and', 'had', 'lived', 'nearly', 'twenty', '-', 'one', 'years', 'in', 'the', 'world', 'with', 'very', 'little', 'to', 'distress', 'or', 'vex', 'her', '.']
[2, 11, 12, 13, 12, 14, 12, 15, 16, 12, 17, 18, 19, 20, 15, 21, 22, 12, 23, 24, 25, 26, 27, 28, 29, 30, 27, 31, 32, 15, 33, 34, 35, 36, 37, 38, 39, 40, 28, 41, 17, 42, 43, 24, 44, 45, 46, 47, 48]
['She', 'was', 'the', 'youngest', 'of', 'the', 'two', 'daughters', 'of', 'a', 'most', 'affectionate', ',', 'indulgent', 'father', ';', 'and', 'had', ',', 'in', 'consequence', 'of', 'her', 'sister', "'", 's', 'marriage', ',', 'been', 'mistress', 'of', 'his', 'house', 'from', 'a', 'very'

In [7]:
print('#Words: ',len(word_dict))
print('#Sentences: ',len(int_sents))
print('Length: ', longest)

#Words:  2728
#Sentences:  1000
Length:  172


Again, we have to pad our sequences that are not long enough, adding 0's at the begining of each ``int_sents`` (although not as necessary in this case):

In [8]:
int_sents = sequence.pad_sequences(int_sents, maxlen=longest)

In [9]:
# an example
int_sents[2]

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
       10,  9])

Consider each input vector contains 30 words, and the corresponding output is the 31st word.

In [10]:
tt = [0,1,2,3,4,5,6]
print('input is', tt[1:1+3])
print('output is',tt[1+3])

input is [1, 2, 3]
output is 4


Let's create our data:

In [11]:
X = []
y = []
window = 30

# Go through the sentences and store the window (sequence of words) as X
# y contains the word at the end of the window
for sent in int_sents:
    for i in range(0, len(sent) - window, 1):
        window_x = sent[i:i + window]
        window_y = sent[i + window]
        X.append(window_x)
        y.append(window_y)  

In [12]:
# For example
sent = int_sents[10]
i= len(sent) - window -1 #since range() creates until len(sent) - window -1
print('input is', sent[i:i + window])
print('output is',sent[i + window])

input is [  0   0   0   0   0   0   0 145 166  12 167  12  50 168 169 170 171  12
 172 129 173 174   3 121 175 176  88 177  17  47]
output is 48


Now we are going to store our data in the correct shape (#sentences x length window x #features (1 - the words)):

In [13]:
X = np.reshape(X, (len(X), window, 1))

# For the y-value, we use one-hot encoding
y = np_utils.to_categorical(y)    

print(np.shape(X))
print(np.shape(y))

(142000, 30, 1)
(142000, 2729)


In [14]:
# For example, although many vectors are 0's since we only consider a dictionary of size 30
ind = 12345
print(X[ind])
print(y[ind])

[[  0]
 [  0]
 [  0]
 [  0]
 [681]
 [578]
 [157]
 [332]
 [ 27]
 [136]
 [682]
 [ 27]
 [680]
 [490]
 [334]
 [102]
 [254]
 [683]
 [ 12]
 [  9]
 [ 77]
 [ 63]
 [ 40]
 [209]
 [684]
 [ 17]
 [501]
 [685]
 [ 32]
 [103]]
[0. 0. 0. ... 0. 0. 0.]


Creating training and test sets:

In [15]:
X = X[:2000]
y = y[:2000]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [16]:
np.shape(y_test)

(660, 2729)

Build the model:

Note:

- check ``keras.LSTM`` configurations here https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTM

- ``Dense`` implements the operation: ``output = activation(dot(input, kernel) + bias)``

In [17]:
model = Sequential()
no_dim = 128 # # of blocks, related to the training time

model = Sequential()
# Input is: a window of 1 feature: an integer representing the word
model.add(LSTM(no_dim, input_shape=(window, 1))) 
model.add(Dropout(0.2))
# The output layer predicts the word, one-hot encoded (i.e. the vector is as long as the number of words)
model.add(Dense(y.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy','mae'])
model.fit(X_train, y_train, validation_split=0.2, batch_size=longest, epochs=20)

Epoch 1/20
7/7 [==============================] - 2s 94ms/step - loss: 7.8807 - accuracy: 0.8069 - mae: 7.3259e-04 - val_loss: 7.7964 - val_accuracy: 0.7910 - val_mae: 7.3256e-04
Epoch 2/20
7/7 [==============================] - 0s 47ms/step - loss: 7.5048 - accuracy: 0.8134 - mae: 7.3244e-04 - val_loss: 6.3587 - val_accuracy: 0.7948 - val_mae: 7.3144e-04
Epoch 3/20
7/7 [==============================] - 0s 44ms/step - loss: 5.5951 - accuracy: 0.8162 - mae: 7.2906e-04 - val_loss: 4.5831 - val_accuracy: 0.7985 - val_mae: 7.2083e-04
Epoch 4/20
7/7 [==============================] - 0s 46ms/step - loss: 3.6749 - accuracy: 0.8134 - mae: 6.9333e-04 - val_loss: 2.6007 - val_accuracy: 0.7948 - val_mae: 5.9401e-04
Epoch 5/20
7/7 [==============================] - 0s 46ms/step - loss: 1.9311 - accuracy: 0.8134 - mae: 4.6644e-04 - val_loss: 1.6454 - val_accuracy: 0.7948 - val_mae: 2.8577e-04
Epoch 6/20
7/7 [==============================] - 0s 47ms/step - loss: 1.3088 - accuracy: 0.8116 - mae: 2

In [18]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128)               66560     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 2729)              352041    
                                                                 
Total params: 418,601
Trainable params: 418,601
Non-trainable params: 0
_________________________________________________________________


Evaluation:

In [19]:
evaluation = model.evaluate(X_test, y_test,return_dict = True)
print(evaluation)

21/21 [==============================] - 0s 6ms/step - loss: 1.4402 - accuracy: 0.8182 - mae: 1.5227e-04
{'loss': 1.4402086734771729, 'accuracy': 0.8181818127632141, 'mae': 0.000152268708916381}


Predicting the words:

In [20]:
no_words = 0
for x_i, y_i in zip(X_test, y_test):
    
    # We need to reshape the test again
    x = np.reshape(x_i, (1, len(x_i), 1))
    
    sentence = ''
    for word in x[0]:
        if np.sum(word) > 0:
            sentence += reverse_dict[word[0]] + " "
    if sentence != '':
        print('\nTo predict: ', sentence)
    
    prediction = model.predict(x)
    
    # The LSTM returns a probability for every word, we take the highest probability (argmax)
    i_x = np.argmax(prediction)
    i_y = np.argmax(y_i)
    if i_x > 0:
        print('Prediction: ', reverse_dict[i_x])
        print('Actual word: ', reverse_dict[i_y])
    no_words += 1
    if no_words > 100:
        break


To predict:  had been living together as friend and friend very mutually attached , and Emma doing just what she liked ; highly esteeming Miss Taylor ' s judgment , but directed 
Prediction:  of
Actual word:  chiefly

To predict:  She was the 

To predict:  It was on the wedding - day of this beloved friend that Emma first sat 
Prediction:  of
Actual word:  in

To predict:  Sixteen years had 

To predict:  Her mother had died too long ago for her to have more than an indistinct remembrance 
Prediction:  of
Actual word:  of

To predict:  of governess , the mildness of her temper had hardly allowed her to impose any restraint ; and the shadow of authority being now long passed away , they had 
Prediction:  of
Actual word:  been

To predict:  Sorrow came -- a gentle sorrow -- but not at all in the shape of 
Prediction:  of
Actual word:  any

To predict:  She was the youngest of the two 
Prediction:  ,
Actual word:  daughters

To predict:  It was on the wedding - day of this beloved frien

You may check `keras.layers.LSTM`'s documentation for more details: 
https://keras.io/api/layers/recurrent_layers/lstm/

Or this Youtube video for more details on RNN model building using  `keras`
https://www.youtube.com/watch?v=BSpXCRTOLJA